In [3]:
import json
from pathlib import Path

In [13]:
JSONDIR = "/tmp/reassemble/"
TSVDIR = "/tmp/word-tsv"

In [14]:
JSONPATH = Path(JSONDIR)
TSVPATH = Path(TSVDIR)

In [15]:
if not TSVPATH.is_dir():
    TSVPATH.mkdir()

In [26]:
def read_json(filename):
    out = []
    with open(filename) as inf:
        data = json.load(inf)
    for seg in data["segments"]:
        out.append({
            "start": seg["start"],
            "end": seg["end"],
            "word": seg["text"]
        })
    return out

In [31]:
filepieces = {}
for file in JSONPATH.glob("*.json"):
    filestem = file.stem
    if not filestem.startswith("hsi_"):
        continue
    pieces = filestem.split("_")
    if len(pieces) != 8:
        continue
    base = "_".join(pieces[0:6])
    if not base in filepieces:
        filepieces[base] = []

    startint = int(pieces[6])
    endint = int(pieces[7])
    words = read_json(str(file))
    for word in words:
        start = (float(startint) / 1000.0) + word["start"]
        end = (float(startint) / 1000.0) + word["end"]
        filepieces[base].append((start, end, word["word"]))

In [32]:
for piece in filepieces:
    pieces = filepieces[piece]
    pieces_sorted = sorted(pieces, key=lambda x: x[0])
    outfile = TSVPATH / f"{piece}.tsv"
    with open(outfile, "w") as of:
        for item in pieces_sorted:
            of.write(f"{item[0]}\t{item[1]}\t{item[2]}\n")